# GIST Driver
##Test on events of interest:
  
  texnet2023vqsh

  
Assumes InjectionSQLScheduled completed

#Configuration

##Libraries

In [0]:
! pip install geopandas
! pip install geodatasets
! pip install folium matplotlib mapclassify contextily

  Using cached geopandas-0.14.3-py3-none-any.whl (1.1 MB)
  Using cached fiona-1.9.6-cp39-cp39-manylinux2014_x86_64.whl (15.7 MB)
  Using cached shapely-2.0.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.5 MB)
  Using cached pyproj-3.6.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (8.3 MB)
  Using cached cligj-0.7.2-py3-none-any.whl (7.1 kB)
  Using cached click_plugins-1.1.1-py2.py3-none-any.whl (7.5 kB)
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-dc814c59-cf20-41b1-854a-87c8b90ec930/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 62 kB 1.0 MB/s 
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-dc814c59-cf20-41b1-854a-87c8b90ec930/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 100 kB 4.7 MB/s 
     |████████████████████████████████| 81 kB 72 kB/s 
  

*** WARNING: max output size exceeded, skipping outpu

In [0]:
import sys
basePath='/Workspace/Users/bill.curry@exxonmobil.com'
gistPath='/Workspace/Users/bill.curry@exxonmobil.com/gist'
sys.path.append(basePath)
import dataBricksConfig as db

In [0]:
import numpy as np
import pandas as pd
import os
import gistMCLive as gi
import eqSQL as es
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas
import contextily as cx

/databricks/python_shell/dbruntime/PythonPackageImportsInstrumentation/__init__.py:171: UserWarning: A NumPy version >=1.22.4 and <2.3.0 is required for this version of SciPy (detected version 1.21.5)
  original_result = python_builtin_import(name, globals, locals, fromlist, level)


In [0]:
import gistPlots as gp

## Parameters

### Input Files
Produced from injectionSQLScheduled.py

In [0]:
injPath=gistPath+'/ScheduledOutput'

DeepWellFile=injPath+'/deep.csv'
DeepInjFile=injPath+'/deepReg.csv'
ShallowWellFile=injPath+'/shallow.csv'
ShallowInjFile=injPath+'/shallowReg.csv'
runPath=gistPath+'/Outputs/'

################################################
# Output prefix for realizations of parameters #
################################################
RealizationPrefix=runPath+'MC'

###General

In [0]:
######################
# General Parameters #
######################
nRealizations=50
nTimeBins=21
WaterDensity=1020.
WaterViscosity=0.001
VerticalCompressibility=0.000000001
FluidCompressibility=0.00000000036
FaultFrictionCoeff=0.6
RockFrictionCoeff=0.6

###################################
# General Parameter Uncertainties #
###################################
WaterDensityUncertainty=5.
WaterViscosityUncertainty=0.000001
VerticalCompressibilityUncertainty=0.00000000001
FluidCompressibilityUncertainty=0.000000000001
FaultFrictionCoeffUncertainty=0.05
RockFrictionCoeffUncertainty=0.05

### Deep Interval

In [0]:
###############################
# Deep Interval Parameters #
###############################
DeepPorosityPercent=8.
# Pulse pressure test shows background diffusivity of 0.5 m2/s
# Back that into permeaiblity: k = D * eta * (Cr + porosity Cp)
# 0.5 x 0.001 x (1e -9 + 0.08 x 3.6 e-10) = 5.144 e-13
# / 9.9e-16 m2 to mD = 520mD!
DeepPermMD=100.
# Now let's change this to something where 1D is the high range and 10mD is the low range

DeepThicknessFT=1400.
# Roughly double of shallow - 
DeepShearModulus=5e9
DeepPoissonsRatioDrained=0.30
DeepPoissonsRatioUndrained=0.31
DeepBiotsCoefficient=0.31

##################################
# DeepInterval Uncertainties #
##################################
DeepPorosityPercentUncertainty=5.
DeepPermLogMDUncertainty=1.
DeepThicknessFTUncertainty=200.
DeepShearModulusUncertainty=1e9
DeepPoissonsRatioDrainedUncertainty=0.005
DeepPoissonsRatioUndrainedUncertainty=0.005
DeepBiotsCoefficientUncertainty=0.05

In [0]:
# Add deep and shallow permeability anisotropy 
# Deep
DeepPermMDSlow=10.
DeepFastSlowRatio=100.
DeepFastAzimuthDeg=30.
DeepOffDiagPermRatio=0.
DeepPermMDSlowLogUnc=0.5
DeepFastSlowLogUnc=1.
DeepFastAzimuthDegUnc=20.
DeepkOffDiagRatioUnc=0.
# Shallow
ShallowPermMDSlow=3.
ShallowFastSlowRatio=10.
ShallowFastAzimuthDeg=-30.
ShallowOffDiagPermRatio=0.5
ShallowPermMDSlowLogUnc=0.5
ShallowFastSlowLogUnc=1.
ShallowFastAzimuthDegUnc=20.
ShallowkOffDiagRatioUnc=0.1


#Earthquake Selection

In [0]:
eqs=es.eqSQL()
EQDF=eqs.getEarthquake('texnet2023vqsh')
EQDF=EQDF.rename(columns={'LatitudeErrorKm':'LatitudeError','LongitudeErrorKm':'LongitudeError','EventId':'EventID'})
# What about the other fault plane solution?
EQDF['Strike']=216.
EQDF['Dip']=65.
EQDF['Rake']=-127.
# We could make use of the Rake
EQDF.info()
eq=EQDF.loc[0]
EQDF.to_csv(runPath+'/11-4-23EQ.csv')

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 35 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   SeismicEventId              1 non-null      int64         
 1   DataSource                  1 non-null      object        
 2   DataSourceUrl               0 non-null      object        
 3   EventID                     1 non-null      object        
 4   EventTimeUtc                1 non-null      datetime64[ns]
 5   EventTimeInLocalTimeZone    1 non-null      object        
 6   EventTimeZone               1 non-null      object        
 7   EventType                   1 non-null      object        
 8   DepthKm                     1 non-null      float64       
 9   DepthErrorKm                1 non-null      float64       
 10  Magnitude                   1 non-null      float64       
 11  MagnitudeError              0 non-null      float64   

#GIST Initialization

## Deep

In [0]:
##################################################
# Initialize 2nd GIST instance for deep interval #
##################################################
deepM=gi.gistMC(nReal=nRealizations,
                ntBin=nTimeBins)
##################################################################
# Initialize pore pressure model                                 #
##################################################################
deepM.initPP(rho_0=WaterDensity,
             phi=DeepPorosityPercent,
             kMD=DeepPermMD,
             h=DeepThicknessFT,
             alphav=VerticalCompressibility,
             beta=FluidCompressibility)

deepM.initMCPP(rhoUnc=WaterDensityUncertainty,
               ntaUnc=WaterViscosityUncertainty,
               phiUnc=DeepPorosityPercentUncertainty,
               kMDLogUnc=DeepPermLogMDUncertainty,
               hUnc=DeepThicknessFTUncertainty,
               alphavUnc=VerticalCompressibilityUncertainty,
               betaUnc=FluidCompressibilityUncertainty)
######################################################
# These parameters need more vetting for Tarzan area #
######################################################
deepM.initPE(mu=DeepShearModulus,
             nu=DeepPoissonsRatioDrained,
             nu_u=DeepPoissonsRatioUndrained,
             alpha=DeepBiotsCoefficient,
             mu_f=FaultFrictionCoeff,
             mu_r=RockFrictionCoeff,
             match=True)
deepM.initMCPE(muUnc=DeepShearModulusUncertainty,
               nuUnc=DeepPoissonsRatioDrainedUncertainty,
               nu_uUnc=DeepPoissonsRatioUndrainedUncertainty,
               alphaUnc=DeepBiotsCoefficientUncertainty,
               muFUnc=FaultFrictionCoeffUncertainty,
               muRUnc=RockFrictionCoeffUncertainty,
               match=True)
deepM.initPPAniso(kMDSlow=DeepPermMDSlow,
                     kFastkSlow=DeepFastSlowRatio,
                     azimuthDeg=DeepFastAzimuthDeg,
                     kOffDiagRatio=DeepOffDiagPermRatio)
###############
# Uncertainty #
###############
deepM.initMCPPAniso(kMDSlowLogUnc=DeepPermMDSlowLogUnc,
                       kFastkSlowLogUnc=DeepFastSlowLogUnc,
                       azimuthDegUnc=DeepFastAzimuthDegUnc,
                       kOffDiagRatioUnc=DeepkOffDiagRatioUnc)

deepM.writeRealizations(RealizationPrefix+'deepAniso.csv')
######################
# Post diffusivities #
######################
print("Deep Diffusivity="+str(deepM.diffPPMax))

  logSpace: lower bound:  10.0 , upper bound:  1000.0
  logSpace: lower bound:  3.1622776601683795 , upper bound:  31.622776601683793
  logSpace: lower bound:  1.0 , upper bound:  1000.0
Deep Diffusivity=0.9405338972348787


#Well selection

####Input wells

In [0]:
#########################################
# Add potential wells to each instance  #
#########################################

In [0]:
print("Before adding deep wells")
deepM.addWells(DeepWellFile,DeepInjFile,verbose=1)
EQDF.info()

Before adding deep wells
 gistMC.addWells: well file added with  2137  wells
 gistMC.addWells: well columns: Index(['ID', 'InjectionWellId', 'APINumber', 'UICNumber', 'Basin', 'County',
       'SurfaceHoleLatitude', 'SurfaceHoleLongitude', 'WellName', 'WellNumber',
       'InjectionType', 'CompletedWellDepthClassification', 'InjectionStatus',
       'CompletionAndDrillingPermitStatus', 'StartDate', 'PermitIsAmendment',
       'DaysToPermitApproval', 'PermittedMaxLiquidBPD',
       'PermittedMaxLiquidPSIG', 'PermittedIntervalBottomFt',
       'PermittedIntervalTopFt', 'PermittedCommercialStatus',
       'PermittedWellDepthClassification', 'Operator', 'OperatorType',
       'OperatorPrincipalCompany', 'OperatorPrincipalCompanyType',
       'PermitStage', 'SRAOrSIR', 'UploadTime'],
      dtype='object')
 gistMC.addWells: injection file has  1792  unique wells
 gistMC.addWells: injection file first day:  3880.0
 gistMC.addWells: injection file last day:  19830.0
 gistMC.addWells: injection

In [0]:
deepPPWells,deepPPExcludedWells,deepPPInj=deepM.findWells(eq,PE=False)
deepPEWells,deepPEExcludedWells,deepPEInj=deepM.findWells(eq,PE=True)
deepPPWells['EventID']=eq['EventID']
deepPEWells['EventID']=eq['EventID']
#deepPPWells.to_csv(runPath+'/Stanton11-4-23_DeepPPWells.csv')
#deepPEWells.to_csv(runPath+'/Stanton11-4-23_DeepPEWells.csv')
#deepPPInj.to_csv(runPath+'/Stanton11-4-23_DeepPPInj.csv')

In [0]:
eq.loc['Origin Date']=pd.to_datetime(eq['Origin Date'])

<command-2066215307979097>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eq.loc['Origin Date']=pd.to_datetime(eq['Origin Date'])


In [0]:
#import gistPlots as gp

#gp.wellInjPlot(deepPPInj,deepPPWells,'01-01-2018','02-11-2023','Deep Injection',True)
#gp.wellInjPlot(shallowPPInj,shallowPPWells,'01-01-2018','02-11-2023','Shallow Injection',True)

#fig, ax = plt.subplots(figsize=(12,24))
# Merge well information onto injection rates
# Need to incorporate time sampling other than 10 day


#Initial GIST runs


In [0]:
# Run deep scenarios
#eq['Origin Date']=pd.to_datetime(eq['Origin Date'])
dPPTimeSeriesAniso,dWellPPAtEQ=deepM.runPressureScenariosAniso(eq,deepPPWells,deepPPInj,verbose=1)

runPressuresAniso: Through setup
runPressuresAniso: Before exp1
runPressuresAniso: After exp1 (561, 1597)
runPressuresAniso: Before sum 561
runPressuresAniso: Time Step  0  of  1596
runPressuresAniso: Time Step  1  of  1596
runPressuresAniso: Time Step  2  of  1596
runPressuresAniso: Time Step  3  of  1596
runPressuresAniso: Time Step  4  of  1596
runPressuresAniso: Time Step  5  of  1596
runPressuresAniso: Time Step  6  of  1596
runPressuresAniso: Time Step  7  of  1596
runPressuresAniso: Time Step  8  of  1596
runPressuresAniso: Time Step  9  of  1596
runPressuresAniso: Time Step  10  of  1596
runPressuresAniso: Time Step  11  of  1596
runPressuresAniso: Time Step  12  of  1596
runPressuresAniso: Time Step  13  of  1596
runPressuresAniso: Time Step  14  of  1596
runPressuresAniso: Time Step  15  of  1596
runPressuresAniso: Time Step  16  of  1596
runPressuresAniso: Time Step  17  of  1596
runPressuresAniso: Time Step  18  of  1596
runPressuresAniso: Time Step  19  of  1596
runPressur

In [0]:
dPPTimeSeriesAniso.shape

Out[17]: (561, 50, 1596)

In [0]:

baseOutPath='/Workspace/Users/bill.curry@exxonmobil.com/gist/Outputs/Stanton11-4-23_v2_'
np.save(baseOutPath+'deepPPAniso.npy',dWellPPAtEQ)
#np.save(baseOutPath+'deepPPTimesAniso.npy',dPPTimeSeriesAniso)
dPPTimeSeriesAnisoSum=dPPTimeSeriesAniso.sum(axis=0)
dPPTimes=np.linspace(deepM.injOT, deepM.injOT+deepM.injDT*(deepM.injNT-1.),deepM.injNT)
np.save(baseOutPath+'deepPPTimesAniso.npy',dPPTimes)
np.save(baseOutPath+'deepPPTimeSeriesAnisoSum.npy',dPPTimeSeriesAnisoSum)

In [0]:
# dPPTimeSeries is time, well, realization - need top wells with time series of pressures
dPPWellIndices=deepPPWells['ID'].isin(deepWindowedWells)
print(sum(dPPWellIndices),' wells out of ',len(dPPWellIndices),' have >0.1 PSI contribution')
dPPTimeSeriesSelected=dPPTimeSeries[:,dPPWellIndices,:]
dPPSelectedWellNameDF=deepPPWells['WellName'][dPPWellIndices]
print(dPPSelectedWellNameDF.to_list())